# Official public work with private tools

This report is dedicated to the analysis of the data on Hilary Clinton'email. The idea of this project is to understand the email controversy arising from use of private email server for official communiactions during her tenure as Secretary of State hit the headline of the media in 2015.

## Data extraction and exploration

In this section, we extract the data from Kaggle. We use the sql file since there are four csv files. We use the package sqlite to extract the data and also all the already used package for the applied data analysis courses.

In [57]:
import numpy as np
import pandas as pd
import sqlite3 as sql3
from datetime import datetime, date, time
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
# Create a database in RAM
db = sql3.connect(':memory:')
# Creates or opens a file called database with a SQLite3 DB
db = sql3.connect('data/database.sqlite')

We use the ``sqlite3``package to extract the data from our file ``database.sqlite``, now we extract the subset containing the emails which are the equivalent to the four csv files.

In [11]:
#Get a cursor object which is named Emails, Persons, Aliases, and EmailReceivers 
mails_cursor = db.cursor()

In [15]:
mails_cursor.execute("SELECT `name`, `sql` FROM `sqlite_master` WHERE type='table';")
results = mails_cursor.fetchall()
for row in results:
    print(row)

('Emails', 'CREATE TABLE Emails (\n    Id INTEGER PRIMARY KEY,\n    DocNumber TEXT,\n    MetadataSubject TEXT,\n    MetadataTo TEXT,\n    MetadataFrom TEXT,\n    SenderPersonId INTEGER,\n    MetadataDateSent TEXT,\n    MetadataDateReleased TEXT,\n    MetadataPdfLink TEXT,\n    MetadataCaseNumber TEXT,\n    MetadataDocumentClass TEXT,\n    ExtractedSubject TEXT,\n    ExtractedTo TEXT,\n    ExtractedFrom TEXT,\n    ExtractedCc TEXT,\n    ExtractedDateSent TEXT,\n    ExtractedCaseNumber TEXT,\n    ExtractedDocNumber TEXT,\n    ExtractedDateReleased TEXT,\n    ExtractedReleaseInPartOrFull TEXT,\n    ExtractedBodyText TEXT,\n    RawText TEXT)')
('Persons', 'CREATE TABLE Persons (\n    Id INTEGER PRIMARY KEY,\n    Name TEXT)')
('Aliases', 'CREATE TABLE Aliases (\n    Id INTEGER PRIMARY KEY,\n    Alias TEXT,\n    PersonId INTEGER)')
('EmailReceivers', 'CREATE TABLE EmailReceivers (\n    Id INTEGER PRIMARY KEY,\n    EmailId INTEGER,\n    PersonId INTEGER)')


Since we didn't find a way to convert the sqlite database into Pandas dataframe, we will in a first time use the csv files to extract the data.

In [2]:
data_folder = "data/"

In [3]:
#Data extraction from the csv-files
emails = pd.read_csv(data_folder + 'Emails.csv.zip')
persons = pd.read_csv(data_folder + 'Persons.csv')
aliases = pd.read_csv(data_folder + 'Aliases.csv')
email_receivers = pd.read_csv(data_folder + 'EmailReceivers.csv')

In [27]:
emails.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [28]:
persons.head()

,Id,Name
0,1,111th Congress
1,2,AGNA USEMB Kabul Afghanistan
2,3,AP
3,4,ASUNCION
4,5,Alec


In [29]:
aliases.head()

,Id,Alias,PersonId
0,1,111th congress,1
1,2,agna usemb kabul afghanistan,2
2,3,ap,3
3,4,asuncion,4
4,5,alec,5


In [30]:
email_receivers.head()

,Id,EmailId,PersonId
0,1,1,80
1,2,2,80
2,3,3,228
3,4,3,80
4,5,4,80


## Data cleaning and handling 

This might be the most longest part, we need to clean the dataset. We will try to capture missing data, we will need to deal with the ``Emails`` dataframe and especially with the rawText column by removing common words, lowering, stemming.

The ``RawText``columns contains text which represents each emails from Clinton's mail.

First, we analyze the type of each column and the size of each dataset before cleaning the data.

In [8]:
print("emails:", emails.shape)
print("persons:", persons.shape)
print("aliases:", persons.shape)
print("email receivers:", email_receivers.shape)

emails: (7945, 22)
persons: (513, 2)
aliases: (513, 2)
email receivers: (9306, 3)


The data frame ``emails `` contains the most features, in fact, it contains all the informations about the emails with 22 features, the ``email_receivers`` has three features and the other only two. Let's also analyze the type of each columns for each frame. 

In [9]:
emails.dtypes

Id                                int64
DocNumber                        object
MetadataSubject                  object
MetadataTo                       object
MetadataFrom                     object
SenderPersonId                  float64
MetadataDateSent                 object
MetadataDateReleased             object
MetadataPdfLink                  object
MetadataCaseNumber               object
MetadataDocumentClass            object
ExtractedSubject                 object
ExtractedTo                      object
ExtractedFrom                    object
ExtractedCc                      object
ExtractedDateSent                object
ExtractedCaseNumber              object
ExtractedDocNumber               object
ExtractedDateReleased            object
ExtractedReleaseInPartOrFull     object
ExtractedBodyText                object
RawText                          object
dtype: object

In [10]:
persons.dtypes

Id       int64
Name    object
dtype: object

In [11]:
aliases.dtypes

Id           int64
Alias       object
PersonId     int64
dtype: object

In [12]:
email_receivers.dtypes

Id          int64
EmailId     int64
PersonId    int64
dtype: object

Since ``emails`` is the biggest dataframe, then it contains most of the information, we will then focus first on this dataset with the handling and the cleaning. Since we have a lot of features, there might be redundant one that we can remove from our dataset. The first thing to do is to convert the date in a proper type by using the package ``datetime``.

In [11]:
#Get the name of each features.
emails.columns

Index(['Id', 'DocNumber', 'MetadataSubject', 'MetadataTo', 'MetadataFrom',
       'SenderPersonId', 'MetadataDateSent', 'MetadataDateReleased',
       'MetadataPdfLink', 'MetadataCaseNumber', 'MetadataDocumentClass',
       'ExtractedSubject', 'ExtractedTo', 'ExtractedFrom', 'ExtractedCc',
       'ExtractedDateSent', 'ExtractedCaseNumber', 'ExtractedDocNumber',
       'ExtractedDateReleased', 'ExtractedReleaseInPartOrFull',
       'ExtractedBodyText', 'RawText'],
      dtype='object')

In [18]:
#Conversion of the date type with the datetime package.
emails['MetadataDateSent'] = emails['MetadataDateSent'].map(pd.to_datetime)
emails['MetadataDateReleased'] = emails['MetadataDateReleased'].map(pd.to_datetime)
emails['ExtractedDateReleased'] = emails['ExtractedDateReleased'].map(pd.to_datetime)

In [14]:
emails.dtypes

Id                                       int64
DocNumber                               object
MetadataSubject                         object
MetadataTo                              object
MetadataFrom                            object
SenderPersonId                         float64
MetadataDateSent                datetime64[ns]
MetadataDateReleased            datetime64[ns]
MetadataPdfLink                         object
MetadataCaseNumber                      object
MetadataDocumentClass                   object
ExtractedSubject                        object
ExtractedTo                             object
ExtractedFrom                           object
ExtractedCc                             object
ExtractedDateSent                       object
ExtractedCaseNumber                     object
ExtractedDocNumber                      object
ExtractedDateReleased           datetime64[ns]
ExtractedReleaseInPartOrFull          category
ExtractedBodyText                       object
RawText      

We can also convert the feature ``ExtractedReleaseInPartOrFull``into a categorical variable since it contains only two values which are ``RELEASE IN FULL`` and ``RELEASE IN PART``.

In [10]:
#Conversion into categorical type.
emails['ExtractedReleaseInPartOrFull'] = emails['ExtractedReleaseInPartOrFull'].astype('category')

In [15]:
emails.dtypes

Id                                       int64
DocNumber                               object
MetadataSubject                         object
MetadataTo                              object
MetadataFrom                            object
SenderPersonId                         float64
MetadataDateSent                datetime64[ns]
MetadataDateReleased            datetime64[ns]
MetadataPdfLink                         object
MetadataCaseNumber                      object
MetadataDocumentClass                   object
ExtractedSubject                        object
ExtractedTo                             object
ExtractedFrom                           object
ExtractedCc                             object
ExtractedDateSent                       object
ExtractedCaseNumber                     object
ExtractedDocNumber                      object
ExtractedDateReleased           datetime64[ns]
ExtractedReleaseInPartOrFull          category
ExtractedBodyText                       object
RawText      

In [16]:
emails.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12 04:00:00,2015-05-22 04:00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,2015-05-13,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03 05:00:00,2015-05-22 04:00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,2015-05-13,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12 04:00:00,2015-05-22 04:00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,2015-05-14,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12 04:00:00,2015-05-22 04:00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,2015-05-13,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11 05:00:00,2015-05-22 04:00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,2015-05-13,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


We need also to convert the features ``ExtractedDataSent``into a datetime variables, since it does not have the same defintion as the other date variables, we need to bring it to the same definition as the other date.

In [51]:
emails['ExtractedDateSent'].values

array(['Wednesday, September 12, 2012 10:16 AM', nan,
       'Wednesday, September 12, 2012 11:52 AM', ...,
       'Thursday, December 16, 2010 10:52 PM', '12/14/201',
       'Friday, December 17, 2010 10:42 AM'], dtype=object)

In [39]:
'2012-'+'09-' + '12' + ' ' '10:'+'16:'+'00'

'2012-09-12 10:16:00'

In [55]:
emails['ExtractedDateSent'].values[10]

'Sunday, March 13, 2011 10:55 AM'

In [46]:
#Conversion of month's name into number.
def month_string_to_number(string):
    '''Conversion of month's name into number
    Args: string which is a month's name
    Returns: an integer which is the month number
    '''
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
         'may':5,
         'jun':6,
         'jul':7,
         'aug':8,
         'sep':9,
         'oct':10,
         'nov':11,
         'dec':12
        }
    s = string.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')

In [47]:
month_string_to_number('September')

9

We need a method to convert the features in the same definitions as the other date in the frame and then we have convert all the date in datetime as the other date features.

## Handling text

The next step is to focus on the ``rawText`` feature. This part is dedicated to the pre-processing of this feature and we use the package ``nltk`` to proceed in text mining and language processing. We take the articles on the webiste https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/ (11.11.2018) as reference.

In [68]:
#Create a data frame which contains the emails. 
rawtext = emails[['RawText']]

This feature consists into text which are the emails text of Clinton'mailbox. We will first extract the number of each words in the emails.

In [69]:
rawtext.head()

,RawText
0,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,UNCLASSIFIED\nU.S. Department of State\nCase N...
2,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [70]:
rawtext['word_count'] = rawText['RawText'].apply(lambda x: len(str(x).split(" ")))
rawtext[['RawText', 'word_count']].head()

/anaconda3/envs/ada/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,RawText,word_count
0,UNCLASSIFIED\nU.S. Department of State\nCase N...,102
1,UNCLASSIFIED\nU.S. Department of State\nCase N...,1157
2,UNCLASSIFIED\nU.S. Department of State\nCase N...,127
3,UNCLASSIFIED\nU.S. Department of State\nCase N...,257
4,B6\nUNCLASSIFIED\nU.S. Department of State\nCa...,1229


We compute also the average word length of each mail. This can improve our analysis.

In [82]:
def avg_word(sentence):
    '''compute the average word length of each sentence.
       Args: sentence, a set of strings
       Return : average of world lenght by sentence'''
    words = sentence.split()
    if (len(words) == 0):
        return 'NaN'
    else:
        return (sum(len(word) for word in words)/len(words))

In [83]:
rawtext['avg_word'] = rawtext['RawText'].apply(lambda x: avg_word(x))
rawtext[['RawText','avg_word']].head()

/anaconda3/envs/ada/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,RawText,avg_word
0,UNCLASSIFIED\nU.S. Department of State\nCase N...,6.03968
1,UNCLASSIFIED\nU.S. Department of State\nCase N...,5.29325
2,UNCLASSIFIED\nU.S. Department of State\nCase N...,5.56051
3,UNCLASSIFIED\nU.S. Department of State\nCase N...,5.48418
4,B6\nUNCLASSIFIED\nU.S. Department of State\nCa...,5.31495


We will need also to compute the number of stopwords of each sentence, we will remove them so that, we can extract more easily the topic of the subject.

In [86]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

rawtext['stopwords'] = rawtext['RawText'].apply(lambda x: len([x for x in x.split() if x in stop]))
rawtext[['RawText','stopwords']].head()

/anaconda3/envs/ada/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,RawText,stopwords
0,UNCLASSIFIED\nU.S. Department of State\nCase N...,9
1,UNCLASSIFIED\nU.S. Department of State\nCase N...,402
2,UNCLASSIFIED\nU.S. Department of State\nCase N...,19
3,UNCLASSIFIED\nU.S. Department of State\nCase N...,6
4,B6\nUNCLASSIFIED\nU.S. Department of State\nCa...,398


We can from now extract the number of stopwords for each mails so that we will be able to remove them if necessary for the next step of our analysis. Since we have all these features, we can now move on text pre-processing.

## Text pre-processing

In order to do text mining, we remove some the features we have count from the text so that, we can find the topic from each mails. We will remove transform the mail into lowercase, remove the common words, the punctuation, the stopwords and also the rare words.

In [87]:
#Transform the mails into lower case
rawtext['RawText'] = rawtext['RawText'].apply(lambda x: " ".join(x.lower() for x in x.split()))
rawtext['RawText'].head()

/anaconda3/envs/ada/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    unclassified u.s. department of state case no....
1    unclassified u.s. department of state case no....
2    unclassified u.s. department of state case no....
3    unclassified u.s. department of state case no....
4    b6 unclassified u.s. department of state case ...
Name: RawText, dtype: object

In [88]:
#Punctuation removing
rawtext['RawText'] = rawtext['RawText'].str.replace('[^\w\s]','')
rawtext['RawText'].head()

/anaconda3/envs/ada/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    unclassified us department of state case no f2...
1    unclassified us department of state case no f2...
2    unclassified us department of state case no f2...
3    unclassified us department of state case no f2...
4    b6 unclassified us department of state case no...
Name: RawText, dtype: object

## Network Analysis

## RawText Analysis

## Data Vizualisation

In [ ]:
#To close the connection with the database.
db.close()